<a href="https://colab.research.google.com/github/BryanChauca/PortfolioProjects/blob/main/Calling_API_and_upload_to_database_(AWS)_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the API call and all the information to a database in the cloud (AWS)

In [20]:
!pip install psycopg2
!pip install psycopg2-binary
import psycopg2 as ps
import pandas as pd

In [21]:
def connect_to_db(host_name, dbname, port, username, password):
  try:
      conn = ps.connect(host=host_name, database=dbname, port=port, user=username, password=password)

  except ps.OperationalError as e:
    raise e
  else:
    print('Connected!')
  return conn



In [56]:
host_name = 'databaseyoutube.cak2dhjeabkv.us-east-1.rds.amazonaws.com'
dbname = 'database_yt_test'
port = '5432'
username = 'bryanchaucaf'
password = 'Chaucamemo1!'
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

Connected!


In [28]:
#create table
def create_table(curr):
  create_table_command = ("""CREATE TABLE IF NOT EXISTS videos(
                    video_id VARCHAR(225) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL)""")
  
  curr.execute(create_table_command)

In [46]:
#check if video exist
def check_if_video_exist(curr, video_id):
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
    curr.execute(query, (video_id,))

    return curr.fetchone() is not None #return a single row from the table

In [55]:
#write the insert command

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
  
  
  insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count, comment_count)VALUES(%s,%s,%s,%s,%s,%s);""")

  row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)

  curr.execute(insert_into_videos, row_to_insert)


In [47]:
#update row if video exist
def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count =%s
            WHERE video_id = %s;""")

    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

In [48]:
#updating the db
#handling data for scalability
def update_db(curr, df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'comment_count'])
#check to see if video exist
    for i, row in df.iterrows():
        if check_if_video_exist(curr, row['video_id']):#if video already exist then we will update
            update_row(curr, row['video_id'],row['video_title'], row['view_count'], row['like_count'], row['comment_count'])
        else:
            tmp_df = tmp_df.append(row)   #if the video does not exist we will append to the db
    return tmp_df


In [61]:
def append_from_df_to_db(curr,df):
  for i, row in df.iterrows():
    insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])

In [67]:
df = pd.read_csv('youtube_1st_job_pull.csv', index_col=0)
df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,0oCTKn_tJ1U,What does a Clinical Data Analyst do,2021-04-06,11893,276,19
1,qcQoBXa0uZ8,What does a Healthcare Data Analyst Do,2021-03-30,32019,722,31
2,dO-F-DPKPpE,What&#39;s on my desk? Desk Tour 2021,2021-03-23,1174,45,6
3,cywuo3Jv0U0,Google Data Analytics Professional Certificati...,2021-03-16,19471,652,96
4,R5P3mN_IN10,How to Learn SAS Programming,2021-03-02,8414,160,17
5,67WmcW71f80,5 Data Analyst Tools for Non Programmers,2021-02-23,3075,129,11
6,mpdcu3V1jiI,3 Reasons to Practice Agile Analytics,2021-02-16,1618,61,3
7,tCq9GkQJnuw,Business Analyst Skills to Learn in 2021,2021-02-09,5397,219,14
8,UAW_YQ_zHZU,Data Analyst Certificate Programs - What to Lo...,2021-02-02,2647,130,12
9,g2xCl5FzJ0c,4 Reasons to Learn SAS for Analytics,2021-01-26,6460,148,18


In [62]:
curr = conn.cursor()

In [63]:
create_table(curr)

In [64]:
new_vid_df = update_db(curr, df)

In [65]:
append_from_df_to_db(curr, new_vid_df)

In [66]:
conn.commit()